# COGS 189 Group 12 
*"Investigating the Impact of Short-Form Video Content on Cognitive Focus Using EEG"*


In [5]:
# Install necessary packages (Uncomment the lines below if you need to install them in your env)
%pip install numpy matplotlib scipy mne brainflow scikit-learn

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import mne
from brainflow.board_shim import BoardShim, BrainFlowInputParams
from brainflow.data_filter import DataFilter, FilterTypes
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


Defaulting to user installation because normal site-packages is not writeable
  Using cached mne-1.9.0-py3-none-any.whl (7.4 MB)
  Using cached brainflow-5.16.0-py3-none-any.whl (28.6 MB)
  Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Load EEG Data

In [5]:
def load_eeg_data(filepath):
    """Loads EEG data from a .npy or .edf file"""
    if filepath.endswith('.npy'):
        data = np.load(filepath)
    elif filepath.endswith('.edf'):
        raw = mne.io.read_raw_edf(filepath, preload=True)
        data = raw.get_data()
    else:
        raise ValueError("Unsupported file format. Use .npy or .edf")
    return data

## Plot Visualization

In [6]:
def plot_eeg_data(data, channel=0, sample_rate=250, duration=5):
    """Plots EEG data for a single channel"""
    time = np.arange(0, duration, 1/sample_rate)
    plt.figure(figsize=(12, 4))
    plt.plot(time[:len(data[channel, :sample_rate*duration])], data[channel, :sample_rate*duration])
    plt.xlabel("Time (s)")
    plt.ylabel("EEG Signal (uV)")
    plt.title(f"EEG Data - Channel {channel}")
    plt.show()

## Bandpass Filter

In [7]:
def bandpass_filter(data, lowcut=1.0, highcut=50.0, sample_rate=250, order=4):
    """Applies a bandpass filter to EEG data"""
    nyquist = 0.5 * sample_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=1)


## Feature Extraction

In [8]:
def compute_psd(data, sample_rate=250):
    """Computes the Power Spectral Density (PSD) of EEG data"""
    freqs, psd = signal.welch(data, sample_rate, nperseg=sample_rate*2)
    return freqs, psd

## Prepare Machine Learning Preprocessing
### Prepares EEG data for ML by standardizing and applying PCA

In [9]:
def prepare_ml_data(data):
    """Prepares EEG data for ML by standardizing and applying PCA"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data.T).T  # Normalize channels
    pca = PCA(n_components=5)  # Reduce dimensionality
    data_pca = pca.fit_transform(data_scaled.T)
    return data_pca

# Experimental Design: Cognitive Focus and Short-Form Video Impact
### The data collection phases: Control Recording -> Break -> Attention Test -> Doom Scroll -> Attention Task

Example usage
events
- "Control Recording": (0, 60), 
- "Break": (60, 120), 
- "Attention Test": (120, 180), 
- "Doom Scroll": (180, 240), 
- "Attention Task": (240, 300)}
    
eeg_data = load_eeg_data(filepath) 


segmented_data = segment_experiment_data(eeg_data, events)